In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 350)
  
#set seed to be able to reproduce the results
np.random.seed(4684)

# Part 1- Adjust Cutoff Point

In [3]:
data = pd.read_csv('./dataset/emails.csv')

In [4]:
data.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,clicked
0,8,short_email,generic,9,Thursday,US,3,0
1,33,long_email,personalized,6,Monday,US,0,0
2,46,short_email,generic,14,Tuesday,US,3,0
3,49,long_email,personalized,11,Thursday,US,10,0
4,65,short_email,generic,8,Wednesday,UK,3,0


In [7]:
data.shape

(99950, 8)

In [5]:
#get dummy variables from categorical ones
data_dummy = pd.get_dummies(data, drop_first=True).drop('email_id', axis=1)
  
#split into train and test to avoid overfitting
train, test = train_test_split(data_dummy, test_size = 0.34)
  
#build the model. We choose a RF, but this issues applies to pretty much all models
rf = RandomForestClassifier(n_estimators=50, oob_score=True)
rf.fit(train.drop('clicked', axis=1), train['clicked'])
#let's print OOB confusion matrix
print(pd.DataFrame(confusion_matrix(train['clicked'], rf.oob_decision_function_[:,1].round(), labels=[0, 1])))

       0    1
0  64342  269
1   1342   14


In [6]:
#and let's print test set confusion matrix
print(pd.DataFrame(confusion_matrix(test['clicked'], rf.predict(test.drop('clicked', axis=1)), labels=[0, 1])))

       0    1
0  33126  144
1    706    7


Firstly, let’s make sure we understand the confusion matrix output. The first one is for RF OOB errors and the second one is for the test set.

Top left (where column == 0 and row == 0) are true negatives. These are the events that the model classifies as 0s and are indeed 0s. Expect this number to always be very large, after all it is the easy part of the problem, given how many 0s we have in the dataset

Bottom left (where column == 0 and row == 1) are the events that the model classifies as 0, but are in fact 1s. It is crucial to minimize this as much as possible. If the model classifies everything as 0, this number will be large

Column == 1 and row == 0, these are the events that are 0s, but the model classifies them as 1. At first, this number will be very low. Again, if it is classifying everything as 0, there will be very few events here. As you improve the model, this number will go up. While you should avoid making this number go up too much (after all they are still misclassifications), that’s part of the trade-off. It is fine to see this number increase in order to increase the number of events correctly classified as 1

Column == 1 and row == 1, these are the true positives. Your goal should be increasing this number. And the only way to do it is by decreasing the number of events that the model classifies as 0, but are in fact 1s (column == 0 and row == 1)

To summarizes all this, we can create class 0 and 1 errors. The first one is class error for class 0 and the other one for class 1. If the model classifies everything as majority class, class error for class 0 will be close to 0 (meaning it is perfect), but the other one will be close to 1 (meaning useless). Your goal is to decrease class 1 error, without increasing too much class 0 error

In [8]:
#confusion matrix test set
conf_matrix = pd.DataFrame(confusion_matrix(test['clicked'], rf.predict(test.drop('clicked', axis=1)), labels=[0, 1]))
#class0/1 errors are 1 -  (correctly classified events/total events belonging to that class)
class0_error = 1 - conf_matrix.loc[0,0]/(conf_matrix.loc[0,0]+conf_matrix.loc[0,1])
class1_error = 1 - conf_matrix.loc[1,1]/(conf_matrix.loc[1,0]+conf_matrix.loc[1,1])
  
print(pd.DataFrame( {'test_class0_error':[class0_error],
                        'test_class1_error':[class1_error]
}))

   test_class0_error  test_class1_error
0           0.004328           0.990182


1 Play with model cut-off point

So obviously the models we built above are useless. Our goal is to force the model to classify more events as class 1, even if this means losing overall accuracy.

The simplest way we can try to achieve that is to simply change the model cut-off point. A couple of words about the cut-off point: every model, for each event, returns a probability between 0 and 1. Typically, model default cut-off value is 0.5, meaning that if that probability is lower than 0.5, the model predicts 0, else predicts 1. But you can change that cut-off value. In a RF, for R that probability is simply the proportion of trees that predict 1. So, if my RF has 50 trees and 10 of those predict 1, the final model score will be 0.2. For Python, it is the mean of all the probabilities returned by each tree.

Right now, our model is using the default 0.5 cut-off value. So it feels pretty straightforward that, if we want to increase the number of events classified as class 1, we can just change the cut-off value accordingly. Maybe if we create a rule like: everything >= 0.1 will be class 1, we will improve our class 1 error. So, let’s see how modifying the cut-off point will change my model output.

In [9]:
from sklearn.metrics import roc_curve
  
#get test set predictions as a probability
pred_prob=rf.predict_proba(test.drop('clicked', axis=1))[:,1]
#get false positive rate and true positive rate, for different cut-off points
#and let's save them in a dataset. 
fpr, tpr, thresholds = roc_curve(test['clicked'],pred_prob)
# we will focus on class errors, defined as
# class0_error = fpr= 1-tnr and class1_error = 1 - tpr
error_cutoff=pd.DataFrame({'cutoff':pd.Series(thresholds),
                               'class0_error':pd.Series(fpr),
                               'class1_error': 1 - pd.Series(tpr)
                                })

In [10]:
#let's also add accuracy to the dataset, i.e. overall correctly classified events.
#This is:  tnr * negative samples in the dataset)/total_events_in_the_data_set + (tpr * positives samples in the test set 
error_cutoff['accuracy']=((1-error_cutoff['class0_error'])*sum(test['clicked']==0)+(1-error_cutoff['class1_error'])*sum(test['clicked']==1))/len(test['clicked'])
  
print(error_cutoff)

        cutoff  class0_error  class1_error  accuracy
0     1.960000      0.000000      1.000000  0.979019
1     0.960000      0.000030      1.000000  0.978989
2     0.944000      0.000090      1.000000  0.978931
3     0.900000      0.000120      1.000000  0.978901
4     0.860000      0.000180      1.000000  0.978842
...        ...           ...           ...       ...
1178  0.001429      0.188127      0.674614  0.801666
1179  0.001176      0.188157      0.674614  0.801636
1180  0.001053      0.188218      0.674614  0.801577
1181  0.000909      0.188248      0.674614  0.801548
1182  0.000000      1.000000      0.000000  0.020981

[1183 rows x 4 columns]


Let’s understand this output:

Cutoff are the different cut-off values we are considering. It usually goes from 0 to 1. In this case, it goes from 0 to 0.96, everything above 0.96 (i.e. 0.97,0.98, etc.) will give the same classification as 0.96. The first row (1.96) doesn’t really matter. It just shows what happens when all events are predicted as majority class and python arbitrarily sets it to 1+max(cutoff)

Class0_error is 1 - true negatives/all negative events. This is also called false negative rate. It simply means: of all class 0 events, how many can 0 correctly classify? Then take 1 minus that number

Class1_error 1 - true positives/all positive events. This is also called false positive rate. It simply means: of all class 1 events, how many can 1 correctly classify? Then take 1 minus that number

Accuracy. This is simply the model accuracy, so correctly classified events divided by all events.

The most important thing of this table is understanding the trade-offs as 1 decrease the cut-off value. Decreasing the cut-off value leads to an increase in class0_error (which is bad), but an improvement in class1_error (which is good). The two extremes (when either class0_error = 0 and class1_error = 1 or the other way round) are obviously useless. It means just classifying all events as a given class. The point is finding the best combination of the two class errors.

Also, as we can see, accuracy is largest when the model classifies everything as majority class. That’s obvious. If we have 98% majority class events, we get a great looking 98% accuracy by classifying everything as class 0. But that’s again useless. As we improve class1 error, we will see accuracy decrease. That’s ok, we are willing to accept that (not that we have any other options). But keep this trade-off in mind making sure accuracy doesn’t go down too much.

The most common way to optimize class0 and class1 errors taking into account the trade-off is by maximizing this formula:

(1-class1_error) - class0_error

The maximum value of this is when both class errors are 0, which will never happen in real life. However, at its core, this simply means that I am willing to increase class0_error by a certain number as long as class1_error goes down by a larger number.

Imagine my starting point is class0_error = 0 and class1_error = 1. The formula above returns 1-1-0 = 0, so it is very bad. If I decrease class1_error by 0.1, but class0_error has increased by only 0.05, that number becomes: 1-0.9-0.05 = 0.05, so it went up and am happy. Which makes sense given that the loss on one side was offset by a larger gain on the other side.

In [11]:
#let's check best combination of class0 and class1 errors
error_cutoff['optimal_value'] = 1 - error_cutoff['class1_error'] - error_cutoff['class0_error']
  
print(error_cutoff.sort_values('optimal_value', ascending=False).head())

        cutoff  class0_error  class1_error  accuracy  optimal_value
1172  0.002500      0.186444      0.676017  0.803284       0.137539
1178  0.001429      0.188127      0.674614  0.801666       0.137258
1179  0.001176      0.188157      0.674614  0.801636       0.137228
1180  0.001053      0.188218      0.674614  0.801577       0.137168
1181  0.000909      0.188248      0.674614  0.801548       0.137138


So, according to this, the best cut-off value is ~0.0025. Meaning that all events whose RF predicted probability is >= 0.0025, will be predicted as 1, else 0. That probability is the mean calculated across the probabilities returned by each tree.

Keep in mind that that formula can be useful, but at the end of the day domain knowledge and a strong business sense are much more important that optimizing some formula. For instance, there might be cases in which the cost of a false positive is much larger than the cost of a false negative. In any case, it would be pretty straightforward to modify that formula accordingly.

All that formula is saying is: I am willing to accept a decrease of X in class0_error as long as class1_error improves by more than X. You can quickly modify it. Imagine the cost of class1_error is much larger than class0_error. You could say something like: I am willing to accept a decrease of X in class0_error as long as class1_error improves by at least X/2. Mathematically, this would be like maximizing:

(1-class1_error)*2 - class0_error

So now a decrease in class1_error weighs twice as much as an increase in class0_error.

Finally, let’s check that everything makes sense by going back to our original model and rebuild the confusion matrix with the new cut-off point.

In [13]:
#we already have predicted probabilities from the previous step, i.e.
#pred_prob=rf.predict_proba(test.drop('clicked', axis=1))[:,1]
  
#let's create a 0/1 vector according to the 0.002 cutoff
best_cutoff = error_cutoff.sort_values('optimal_value', ascending=False)['cutoff'].values[0]
predictions=np.where(pred_prob>=best_cutoff,1,0)
#get confusion matrix for those predictions
#confusion matrix test set
conf_matrix_new = pd.DataFrame(confusion_matrix(test['clicked'], predictions, labels=[0, 1]))
print(conf_matrix_new)

       0     1
0  27067  6203
1    482   231


In [14]:
class0_error = 1 - conf_matrix_new.loc[0,0]/(conf_matrix_new.loc[0,0]+conf_matrix_new.loc[0,1])
class1_error = 1 - conf_matrix_new.loc[1,1]/(conf_matrix_new.loc[1,0]+conf_matrix_new.loc[1,1])
print(pd.DataFrame( {'cutoff':[best_cutoff],
                         'test_class0_error_new':[class0_error],
                         'test_class1_error_new':[class1_error]
}))

   cutoff  test_class0_error_new  test_class1_error_new
0  0.0025               0.186444               0.676017


It worked! As we can see, true/false positive/negative rates are exactly the same as in the error_cutoff dataset when the cut-off is 0.0025

Pros and Cons

Pros of using the cut-off value strategy with unbalanced data

✓ The interpretation is very straightforward. Since one class is way less likely to happen than the other one, I just force the model to classify more events as minority class by lowering the classification threshold. You are not modifying the data distribution or, in general, doing any tranformation that leads to lose interpretability

Cons of using the cut-off value strategy with unbalanced data

✓ It is not always possible to use this approach. If our classes are highly unbalanced, our model might not find any way to separate them. That would lead to basically no model and all events being classified as majority class with probability 1. In that case, since all events have probability 1, there is no point in doing a cut-off analysis. More on this in the next section

# Part 2- Apply Weights

In [15]:
import pandas as pd
import numpy as np
import graphviz
from sklearn.ensemble import RandomForestClassifier
np.random.seed(4684)
from numpy.core.umath_tests import inner1d
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from graphviz import Source
from scipy import stats
  
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 350)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  


In [17]:
data = pd.read_csv('./dataset/emails.csv')

In [18]:
#get dummy variables from categorical ones
data_dummy = pd.get_dummies(data, drop_first=True).drop('email_id', axis=1)
  
#split into train and test to avoid overfitting
train, test = train_test_split(data_dummy, test_size = 0.34)
  
#build the model. We choose a single decision tree here, but this issue might apply to all models.
tree=DecisionTreeClassifier(min_impurity_decrease=0.001)
tree.fit(train.drop('clicked', axis=1),train['clicked'])
  
#visualize it
export_graphviz(tree, out_file="tree.dot", feature_names=train.drop('clicked', axis=1).columns, proportion=True, rotate=True)
with open("tree.dot") as f:
    dot_graph = f.read()
s = Source.from_file("tree.dot")
s.view()

'tree.dot.pdf'

No splits at all! Not even on just one variable. So if we check the probability assigned to each event on the test set, we get:

In [19]:
tree_probabilities = tree.predict_proba(test.drop('clicked', axis=1))[:,1]
print(stats.describe(tree_probabilities))

DescribeResult(nobs=33983, minmax=(0.020555732411660376, 0.020555732411660376), mean=0.020555732411660376, variance=0.0, skewness=0.0, kurtosis=-3.0)


That is: min and max are the same! Our model is returning 0.021 probability for every single event in the dataset. This was simply the starting proportion of minority class. In a situation like this, there is obviously no point in optimizing the cut-off point. We will have class0 error = 0 and class1 error = 1 for all cut-offs above 0.0207 and class0 error = 1 and class1 error = 0 for all cut-offs below or equal to 0.021. No other option.

Something like this happens very often when the data are highly unbalanced and/or hard to separate.

Changing weights

If we cannot use the model cut-off point, the next best strategy is to change the weights of the events in your training set. Specifically, you want to increase the weight of the minority class. This will force the model towards correctly predicting more class 1 events. But it comes at the expenses of class 0. So, similarly to when we changed the cut-off point, expect class 1 loss to go down and class 0 loss to go up.

A way to look at why changing weights works is the following:

The main problem we have when building the model is that the accuracy the model would achieve by classifying everything as majority class is very high. So the model has no incentive in trying to correctly predict class 1 events, especially because this will mean misclassifying class 0 events and, therefore, losing accuracy

By changing weights and increasing the number of minority class events, the accuracy coming from classifying everything as majority class drops. The extreme case is that you make the classes balanced, so class 1 and class 0 proportion becomes 50/50. So the model has to now try to separate the classes just like it happens in common ML problems when classes are not heavily unbalanced. After all, if starting class proportion is 50/50, any non-random split will improve that and move accuracy >50%.

Unfortunately, there is no hard rule on how to optimize class weights. In general, an approach similar to the one used for the cut-off point will work. That is, try a few different weights, look at class errors and pick the weight where the trade-off between the two class errors is the best.

In [21]:
# RF weights can be passed as class weights inside RandomForestClassifier. 
#Then, for each weight configuration, we save class errors and accuracy. 
#Finally, pick the best combination of both class errors 
  
#Build 20 RF models with different weights
class0_error = []
class1_error = []
accuracy = []
  
#apply weights from 10 to 200 with 10 as a step
for i in range(10,210,10) :
               rf = RandomForestClassifier(n_estimators=50, oob_score=True, class_weight={0:1,1:i})
               rf.fit(train.drop('clicked', axis=1), train['clicked'])
               #let's get confusion matrix
               conf_matrix = pd.DataFrame(confusion_matrix(test['clicked'], rf.predict(test.drop('clicked', axis=1)), labels=[0, 1]))
               #class0/1 errors are 1 -  (correctly classified events/total events belonging to that class)
               class0_error.append( 1 - conf_matrix.loc[0,0]/(conf_matrix.loc[0,0]+conf_matrix.loc[0,1]))
               class1_error.append( 1 - conf_matrix.loc[1,1]/(conf_matrix.loc[1,0]+conf_matrix.loc[1,1]))
               accuracy.append((conf_matrix.loc[1,1]+conf_matrix.loc[0,0])/conf_matrix.values.sum())
  
dataset_weights = pd.DataFrame ({'minority_class_weight': pd.Series(range(10,210,10)),
                                     'class0_error': pd.Series(class0_error),
                                     'class1_error': pd.Series(class1_error),
                                     'accuracy':     pd.Series(accuracy)
                                   })
  
print(dataset_weights)

    minority_class_weight  class0_error  class1_error  accuracy
0                      10      0.028885      0.959327  0.951593
1                      20      0.047610      0.931276  0.933849
2                      30      0.065434      0.900421  0.917047
3                      40      0.073940      0.900421  0.908719
4                      50      0.078058      0.900421  0.904688
5                      60      0.084100      0.889201  0.899008
6                      70      0.085813      0.886396  0.897390
7                      80      0.088067      0.883590  0.895242
8                      90      0.087677      0.880785  0.895683
9                     100      0.089961      0.882188  0.893417
10                    110      0.089390      0.887798  0.893859
11                    120      0.089991      0.882188  0.893388
12                    130      0.091704      0.877980  0.891799
13                    140      0.092005      0.876578  0.891534
14                    150      0.091133 

Let’s understand this output:

Minority_class_weight is the weight given to the minority class. So 10 means minority class is weighed 10 times more than the majority one, and so on. In our dataset, we had a 2:98 ratio between the classes. That means that when we give the minority class weight ~50, we achieve a balanced dataset, i.e. ~100/98 ratio.

Class0 and class1 error as well as accuracy are as described in the previous section. we can see that, as we increase the weight of the minority class, more and more points get classified as class 1, so class0 error will increase (bad), class1 error will decrease (good), and accuracy will go down (bad). And btw this is exactly the same trade-off we saw when lowering the cut-off. However, at some point those values kind of flatten on the test set.

The formula we previously used to optimize the two class errors, (1-class1_error) - class0_error, can also be used here:

In [22]:
#Calculate trade-ff between class errors
dataset_weights['optimal_value'] = 1 - dataset_weights['class1_error'] - dataset_weights['class0_error']
  
#Order by optimal_value and pick the first row
print(dataset_weights.sort_values('optimal_value', ascending=False).head(1))

   minority_class_weight  class0_error  class1_error  accuracy  optimal_value
2                     30      0.065434      0.900421  0.917047       0.034145


So, according to this, the best weight is 40, meaning we want to increase minority class weight by 40. If we do that, we will get to a ratio between the classes of 80:98 (the starting point was 2:98), pretty close to balanced.

And that’s it. The model is not predicting all events as majority class. And we managed to improve class 1 error, without hurting accuracy and class 0 error too much.

Pros and Cons

Pros of using the the weight strategy with unbalanced data

 It is very flexible. we can try as many different weights as we wish and find out the ones that work best for us. we have way less flexibility with the cut-off approach. And btw we could also use this on top of the cut-off approach, i.e. firstly change weights and then find the best cut-off.

 It is pretty easy to explain it. we can simply say: since we had less class 1 events, we increased their number to force the model to learn about them
 
Cons of using the the weight strategy with unbalanced data

 we risk overfitting if the starting number of minority class events is not large enough. Imagine we have just 10 minority class events. By increasing their weight, we are essentially creating many events just like them. So now we have tons of events which are all the same. The model will correctly classify those, but this will hardly generalize to new data. Changing weights and in general resampling do not work when our sample size is too smal. 

 It can be computationally expensive if we have to try many different weights and build a model for each weight. Making data balanced without specifically looking for the optimal weights is a commonly used short-cut. As with all short-cuts use it with caution though